In [74]:
import os, sys
import numpy as np
import plotly.express as px
sys.path.append('../')

In [75]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [76]:
%autoreload 2
from onsstove.onsstove import OnSSTOVE
from onsstove.layer import RasterLayer, VectorLayer
from onsstove.raster import interpolate
import time

start = time.time()

## 1. Create an OnSSTOVE model

In [77]:
nepal = OnSSTOVE(project_crs=3857, cell_size=(1000, 1000))
output_directory = 'results'
nepal.output_directory = output_directory

## 2. Read the scenario data

In [78]:
path = r"C:\Users\camilorg\Box Sync\EGI Energy Systems\06 Projects\2021 Nepal Geospatial cooking\02 - work\OnSSTOVE cases\NP_test_file_social_specs.csv"
nepal.read_scenario_data(path, delimiter=',')

## 3. Read the cooking technologies data

In [79]:
path = r"C:\Users\camilorg\Box Sync\EGI Energy Systems\06 Projects\2021 Nepal Geospatial cooking\02 - work\OnSSTOVE cases\NP_test_file_tech_specs.csv"
nepal.read_tech_data(path, delimiter=',')

## 4. Add a country mask layer

In [80]:
path = r"C:\Users\camilorg\Box Sync\Clean Cooking Energy\QGIS\Administrative\npl_admbnda_nd_20201117_shp\npl_admbnda_adm0_nd_20201117.shp"
mask_layer = VectorLayer('admin', 'adm_0', layer_path=path)
os.makedirs(f'{output_directory}/admin/adm_0', exist_ok=True)
mask_layer.reproject(3857, f'{output_directory}/admin/adm_0')
nepal.mask_layer = mask_layer
# add admin_1 or 2 or 3 layer to gdf here

## 5. Add a population base layer

In [81]:
path = r"data\population_npl_2018-10-01.tif"
os.makedirs(f'{output_directory}/demographics/population', exist_ok=True)
population = RasterLayer('demographics', 'population', layer_path=path, resample='sum')
population.reproject(3857, f'{output_directory}/demographics/population', 1000, 1000)
population.mask(nepal.mask_layer.layer, f'{output_directory}/demographics/population')
nepal.base_layer = population
nepal.population_to_dataframe(population)

## 6. Calibrate population and urban/rural split

In [82]:
nepal.calibrate_pop()
nepal.calibrate_urban()

## 7. Calculate parameters of base fuel (Biomass)

### 7.1. Health costs

In [83]:
# Calculate relative risk for each disease for the base fuel
rr_0_alri, rr_0_copd, rr_0_ihd, rr_0_lc = nepal.base_fuel.relative_risk()

# Calculate the Population Atributable Factor (PAF) for each disease for the base fuel
paf_0_alri = nepal.base_fuel.paf(rr_0_alri, 1 - nepal.specs['clean_cooking_access'])
paf_0_copd = nepal.base_fuel.paf(rr_0_copd, 1 - nepal.specs['clean_cooking_access'])
paf_0_ihd  = nepal.base_fuel.paf(rr_0_ihd, 1 - nepal.specs['clean_cooking_access'])
paf_0_lc = nepal.base_fuel.paf(rr_0_lc, 1 - nepal.specs['clean_cooking_access'])

### 7.2. Carbon emissions and related costs

In [84]:
nepal.base_fuel.carb(nepal.specs, nepal.gdf)

### 7.3. Time for travelling, collecting fuel, and cooking

In [85]:
# paths to GIS layers
nepal.base_fuel.friction_path = r"C:\Users\camilorg\Box Sync\Clean Cooking Energy\OnSSTOVE\data\2020_walking_only_friction_surface\2020_walking_only_friction_surface.geotiff"
nepal.base_fuel.forest_path = r"C:\Users\camilorg\Box Sync\Clean Cooking Energy\OnSSTOVE\data\Forest cover\Global Forest Cover Change (GFCC).tif"

# Calculate total time
nepal.base_fuel.total_time(nepal)

## 8. Add wealth index GIS data

In [86]:
path = r"C:\Users\camilorg\Box Sync\EGI Energy Systems\06 Projects\2021 Nepal Geospatial cooking\02 - work\GIS-data\Demand\Wealth Index\Wealth index 2011.tif"
wealth_index = RasterLayer('demographics', 'Wealth_index', layer_path=path,  resample='average')
os.makedirs(f'{output_directory}/demographics/Wealth_index', exist_ok=True)
interpolate(wealth_index.path)
wealth_index.align(population.path, f'{output_directory}/demographics/Wealth_index')

### 8.1. Calculate value of time

In [87]:
# Based on wealth index, minimum wage and a lower an upper range for cost of oportunity
nepal.get_value_of_time(wealth_index)

## 9. Read electricity network GIS layers

In [88]:
# Read MV lines
path = r"C:\Users\camilorg\Box Sync\Clean Cooking Energy\QGIS\Power network\Nepal_DL0\Nepal_DL0.shp"
mv_lines = VectorLayer('electricity', 'MV_line', layer_path=path)
os.makedirs(f'{output_directory}/electricity/MV_line', exist_ok=True)
mv_lines.reproject(3857, f'{output_directory}/electricity/MV_line')

# Read HV lines
path = r"C:\Users\camilorg\Box Sync\Clean Cooking Energy\QGIS\Power network\HV-network\HV_lines_georeferenced.shp"
hv_lines = VectorLayer('electricity', 'HV_line', layer_path=path)
os.makedirs(f'{output_directory}/electricity/HV_line', exist_ok=True)
hv_lines.reproject(3857, f'{output_directory}/electricity/HV_line')

### 9.1. Calculate distance to electricity infrastructure 

In [89]:
nepal.distance_to_electricity(hv_lines=hv_lines, mv_lines=mv_lines)

## 10. Add night time lights data

In [90]:
path = r"data\nighttime_lights.tif"
os.makedirs(f'{output_directory}/electricity/Night_lights', exist_ok=True)
ntl = RasterLayer('electricity', 'Night_lights', layer_path=path, resample='average')
ntl.align(population.path, f'{output_directory}/electricity/Night_lights')
ntl.layer[np.isnan(ntl.layer)] = 0 # hey lets make sure that we interpolate NaNs

nepal.raster_to_dataframe(ntl.layer, name='Night_lights', method='read')

## 11. Calibrate current electrified population

In [91]:
nepal.elec_current()

print('Calibrated grid electrified population fraction:', nepal.gdf['Elec_pop_calib'].sum() / nepal.gdf['Calibrated_pop'].sum())

We have identified the existence of transformers or MV lines as input data; therefore we proceed using those for the calibration
Calibrated electrified population fraction: 0.7199999999999999


## 12. Adding GIS data for LPG supply

In [92]:
nepal.techs['LPG'].lpg_path = r"C:\Users\camilorg\Box Sync\EGI Energy Systems\06 Projects\2021 Nepal Geospatial cooking\02 - work\GIS-data\Supply\LPG\Nepal_Gas_12Jun2021_Final4.shp"
nepal.techs['LPG'].friction_path = r"C:\Users\camilorg\Box Sync\Clean Cooking Energy\OnSSTOVE\data\2020_motorized_friction_surface\2020_motorized_friction_surface.geotiff"

nepal.techs['LPG'].add_travel_time(population, output_directory)

## 13. Adding GIS data for Improved Biomass collected

In [93]:
nepal.techs['Collected_Improved_Biomass'].friction_path = r"C:\Users\camilorg\Box Sync\Clean Cooking Energy\OnSSTOVE\data\2020_walking_only_friction_surface\2020_walking_only_friction_surface.geotiff"
nepal.techs['Collected_Improved_Biomass'].forest_path = r"C:\Users\camilorg\Box Sync\Clean Cooking Energy\OnSSTOVE\data\Forest cover\Global Forest Cover Change (GFCC).tif"

## 14. Calculate benefits and costs of each technology

In [94]:
names = ['Electricity', 'Collected_Traditional_Biomass', 'Collected_Improved_Biomass', 'LPG']
techs = [nepal.techs[name] for name in names]

# Loop through each technology and calculate all benefits and costs
for tech in techs:
    print(f'Calculating health benefits for {tech.name}...')
    tech.morbidity(nepal.specs, nepal.gdf, paf_0_alri, paf_0_copd, paf_0_lc, paf_0_ihd)
    tech.mortality(nepal.specs, nepal.gdf, paf_0_alri, paf_0_copd, paf_0_lc, paf_0_ihd)
    print(f'Calculating carbon emissions benefits for {tech.name}...')
    tech.carbon_emissions(nepal.specs, nepal.gdf, nepal.base_fuel.carbon)
    print(f'Calculating time saved benefits for {tech.name}...')
    tech.time_saved(nepal)
    print(f'Calculating costs for {tech.name}...')
    tech.discounted_om(nepal.gdf, nepal.specs)
    tech.discounted_inv(nepal.gdf, nepal.specs)
    tech.discounted_meals(nepal.gdf, nepal.specs)
    tech.discount_fuel_cost(nepal.gdf, nepal.specs, nepal.rows, nepal.cols)
    tech.salvage(nepal.gdf, nepal.specs)
    print(f'Calculating net benefit for {tech.name}...\n')
    tech.net_benefit(nepal.gdf)

Calculating health benefits for Electricity...
Calculating carbon emissions benefits for Electricity...
Calculating time saved benefits for Electricity...
Calculating costs for Electricity...
Calculating net benefit for Electricity...

Calculating health benefits for Collected_Traditional_Biomass...
Calculating carbon emissions benefits for Collected_Traditional_Biomass...
Calculating time saved benefits for Collected_Traditional_Biomass...
Calculating costs for Collected_Traditional_Biomass...
Calculating net benefit for Collected_Traditional_Biomass...

Calculating health benefits for Collected_Improved_Biomass...
Calculating carbon emissions benefits for Collected_Improved_Biomass...
Calculating time saved benefits for Collected_Improved_Biomass...
Calculating costs for Collected_Improved_Biomass...
Calculating net benefit for Collected_Improved_Biomass...

Calculating health benefits for LPG...
Calculating carbon emissions benefits for LPG...
Calculating time saved benefits for LPG

## 15. Getting the max benefit technology for each cell

In [95]:
nepal.maximum_net_benefit()

## 16. Printing the results

In [96]:
nepal.gdf

,geometry,Pop,Calibrated_pop,IsUrban,Households,value_of_time,HV_line_dist,MV_line_dist,Night_lights,Elec_dist,...,benefits_Collected_Traditional_Biomass,net_benefit_Collected_Traditional_Biomass,costs_Collected_Improved_Biomass,benefits_Collected_Improved_Biomass,net_benefit_Collected_Improved_Biomass,costs_LPG,benefits_LPG,net_benefit_LPG,max_benefit_tech,maximum_net_benefit
0,POINT (9090104.530 3543192.836),31.171455,31.344224,0,5.498987,0.032189,58.872746,40.521599,0.000000,40.521599,...,0.0,-0.916498,46.902608,485.900271,438.997663,1471.833688,1712.305464,240.471776,Collected_Improved_Biomass,438.997663
1,POINT (9090104.530 3536192.836),46.757183,47.016337,0,8.248480,0.032323,52.392746,33.837849,0.000000,33.837849,...,0.0,-1.374747,70.353911,729.335081,658.981170,2190.039537,2605.462928,415.423390,Collected_Improved_Biomass,658.981170
2,POINT (9083104.530 3535192.836),5.517799,5.548382,0,0.973400,0.032684,54.708317,35.846897,0.000000,35.846897,...,0.0,-0.162233,8.302441,86.222328,77.919886,258.487731,297.509113,39.021382,Collected_Improved_Biomass,77.919886
3,POINT (9084104.530 3535192.836),15.585728,15.672112,0,2.749493,0.032658,54.203320,35.355339,0.000000,35.355339,...,0.0,-0.458249,23.451304,243.515425,220.064121,729.396677,841.251848,111.855171,Collected_Improved_Biomass,220.064121
4,POINT (9064104.530 3534192.836),15.585728,15.672112,0,2.749493,0.035454,64.536812,47.634022,0.000000,47.634022,...,0.0,-0.458249,23.451304,246.882335,223.431032,748.961096,878.763888,129.802791,Collected_Improved_Biomass,223.431032
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96713,POINT (9797104.530 3045192.836),66.798883,1.257744,0,0.220657,0.055610,16.031219,0.000000,0.173656,0.000000,...,0.0,-0.036776,1.882053,21.761258,19.879205,47.410605,83.678364,36.267759,LPG,0.679115
96715,POINT (9721104.530 3044192.836),164.841528,3.103772,0,0.544521,0.058040,10.049875,0.000000,0.240596,0.000000,...,0.0,-0.090754,4.644396,54.280409,49.636013,115.640852,249.531191,133.890339,LPG,2.507102
96718,POINT (9797104.530 3044192.836),536.486856,10.101416,1,1.772178,0.055515,17.029387,0.000000,0.000000,0.000000,...,0.0,-0.295363,15.115472,174.698725,159.583254,NaN,675.685534,NaN,Collected_Improved_Biomass,2.988202
96720,POINT (9722104.530 3043192.836),642.881959,12.104710,1,2.123633,0.057113,11.000000,0.000000,0.891932,0.000000,...,0.0,-0.353939,18.113145,210.831384,192.718239,NaN,968.323809,NaN,Collected_Improved_Biomass,3.608656


In [97]:
nepal.lives_saved()
nepal.health_costs_saved()
nepal.extract_time_saved()
nepal.reduced_emissions()
nepal.investment_costs()
nepal.fuel_costs()
nepal.emissions_costs_saved()

In [99]:
nepal.gdf.groupby(['max_benefit_tech']).agg({'Calibrated_pop': lambda row: np.nansum(row) / 1000000,
                                                             'maximum_net_benefit': lambda row: np.nansum(row) / 1000000,
                                                             'deaths_avoided': 'sum',
                                                             'health_costs_avoided': lambda row: np.nansum(row) / 1000000,
                                                             'time_saved': 'sum',
                                                             'reduced_emissions': lambda row: np.nansum(row) / 1000000000})

,Calibrated_pop,maximum_net_benefit,deaths_avoided,health_costs_avoided,time_saved,reduced_emissions
max_benefit_tech,,,,,,
Collected_Improved_Biomass,0.355697,5.166439,107.416307,3.524386,2.741873e+07,222.669157
Electricity,20.599200,1144.918224,24926.672230,833.824915,9.598147e+09,22616.696410
LPG,7.655103,171.481340,7951.293712,268.051591,3.237949e+09,7693.236110


In [100]:
print('maximum_net_benefit:', nepal.gdf['maximum_net_benefit'].sum() / 1000000)
print('deaths_avoided:', nepal.gdf['deaths_avoided'].sum())
print('health_costs_avoided:', nepal.gdf['health_costs_avoided'].sum() / 1000000)
print('time_saved:', nepal.gdf['time_saved'].sum() / nepal.gdf['Households'].sum() / 365)
print('reduced_emissions:', nepal.gdf['reduced_emissions'].sum() / 1000000000)
print('investment_costs:', nepal.gdf['investment_costs'].sum() / 1000000)
print('fuel_costs:', nepal.gdf['fuel_costs'].sum() / 1000000)
print('emissions_costs_saved:', nepal.gdf['emissions_costs_saved'].sum() / 1000000)

maximum_net_benefit: 1321.566002715108
deaths_avoided: 32985.38224835494
health_costs_avoided: 1105.4008913464331
time_saved: 6.893663481778198
reduced_emissions: 30.532601677027603
investment_costs: 257.4570347009013
fuel_costs: 620.6274303494092
emissions_costs_saved: 152.663008385138


## 17. Saving data to raster files

In [101]:
nepal.to_raster('max_benefit_tech')
nepal.to_raster('net_benefit_Electricity')
nepal.to_raster('net_benefit_LPG')
nepal.to_raster('net_benefit_Collected_Traditional_Biomass')
nepal.to_raster('net_benefit_Collected_Improved_Biomass')
nepal.to_raster('maximum_net_benefit')
nepal.to_raster('investment_costs')

Layer saved in results\Output\max_benefit_tech.tif

Variable codes:
    Collected_Improved_Biomass: 0
    LPG: 1
    Electricity and Collected_Improved_Biomass: 2
    Electricity and LPG: 3

Layer saved in results\Output\net_benefit_Electricity.tif

Layer saved in results\Output\net_benefit_LPG.tif

Layer saved in results\Output\net_benefit_Collected_Traditional_Biomass.tif

Layer saved in results\Output\net_benefit_Collected_Improved_Biomass.tif

Layer saved in results\Output\maximum_net_benefit.tif

Layer saved in results\Output\investment_costs.tif



In [57]:
nepal.gdf.to_csv(r'C:\Users\camilorg\Box Sync\Clean cooking Africa paper/sample_output.csv')

In [ ]:
end = time.time()

diff = end - start
print('Execution time:', str(str(int(diff//60))) + ' min ' + str(int((diff)%60)) + ' sec')

In [102]:
dff = nepal.gdf.groupby(['max_benefit_tech']).agg({'Calibrated_pop': lambda row: np.nansum(row) / 1000000,
                                                   'maximum_net_benefit': lambda row: np.nansum(row) / 1000000,
                                                   'investment_costs': lambda row: np.nansum(row) / 1000000}).reset_index()

In [67]:
fig = px.pie(dff, names='max_benefit_tech', values='Calibrated_pop', color_discrete_sequence=['#dc0f0f', '#a86ee1', '#79de13'])
fig.write_image(os.path.join(output_directory, 'output', 'Pop_per_tech.pdf'), width=500, height=500)
fig.update_traces(textfont_size=16)
fig.show()

In [103]:
fig = px.pie(dff, names='max_benefit_tech', values='investment_costs', color_discrete_sequence=['#dc0f0f', '#a86ee1', '#79de13'])
fig.write_image(os.path.join(output_directory, 'output', 'Investments.pdf'), width=500, height=500)
fig.update_traces(textfont_size=16)
fig.show()